<a href="https://colab.research.google.com/github/fulltrick/-/blob/main/chilloutmix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#────────────────────────────────────────────
# ChilloutMix Stable Diffusion WebUI - Google Colab版
# リアルな人物画像生成に特化したモデル
#────────────────────────────────────────────

#────────────────────────────────────────────
# 0) 環境のリセットと準備
#────────────────────────────────────────────

from google.colab import drive
import os
import subprocess
import sys
import shutil
import requests
import json

print("🎭 ChilloutMix Stable Diffusion WebUI セットアップ開始")
print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print("🔄 環境をクリーンアップ中...")

# 既存のディレクトリを削除（クリーンスタート）
if os.path.exists('/content/stable-diffusion-webui'):
    shutil.rmtree('/content/stable-diffusion-webui')
    print("✅ 既存のStable Diffusion WebUIディレクトリを削除")

# Google Driveをマウント
try:
    drive.mount('/content/drive')
    print("✅ Google Driveマウント完了")
except:
    print("⚠️ Google Driveは既にマウント済み")

print("\n📝 ChilloutMixについて:")
print("• リアルな人物画像生成に特化したモデル")
print("• アジア系女性の画像生成で高い品質を発揮")
print("• 写実的なポートレート生成が得意")
print("• ファイルサイズ: 約4.27GB")

#────────────────────────────────────────────
# 1) 必要なパッケージをインストール
#────────────────────────────────────────────

print("\n📦 必要なパッケージをインストール中...")
!apt-get update -qq
!apt-get install -y -qq aria2 wget curl git

#────────────────────────────────────────────
# 2) Stable Diffusion Web UI をクローン
#────────────────────────────────────────────

print("⬇️ Stable Diffusion Web UIをクローン中...")
%cd /content
!git clone -q https://github.com/AUTOMATIC1111/stable-diffusion-webui.git

#────────────────────────────────────────────
# 3) PyTorch + CUDA 12.1 をインストール
#────────────────────────────────────────────

print("🔥 PyTorch + CUDA 12.1をインストール中...")
%cd /content/stable-diffusion-webui
!pip install -q --upgrade pip
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

#────────────────────────────────────────────
# 4) 拡張機能をインストール
#────────────────────────────────────────────

print("🔌 拡張機能をインストール中...")
%cd /content/stable-diffusion-webui/extensions

# ControlNet 拡張をインストール
!git clone -q https://github.com/Mikubill/sd-webui-controlnet.git

# 顔修復用拡張（代替手段）
# GFPGANの代わりに、より軽量なCodeFormerを使用
!git clone -q https://github.com/sczhou/CodeFormer.git

# 高画質化拡張
!git clone -q https://github.com/pkuliyi2015/multidiffusion-upscaler-for-automatic1111.git

# タグ補完拡張
!git clone -q https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git

print("✅ 拡張機能のインストール完了")

#────────────────────────────────────────────
# 5) ChilloutMixモデルをダウンロード
#────────────────────────────────────────────

def download_chilloutmix_model():
    """ChilloutMixモデルをダウンロード"""
    print("🎭 ChilloutMixモデルをダウンロード中...")

    # モデル保存ディレクトリを作成
    model_dir = '/content/stable-diffusion-webui/models/Stable-diffusion'
    os.makedirs(model_dir, exist_ok=True)

    # ChilloutMix-Ni (推奨バージョン)
    model_urls = [
        {
            "name": "ChilloutMix-Ni",
            "filename": "chilloutmix_NiPrunedFp32Fix.safetensors",
            "url": "https://huggingface.co/naonovn/chilloutmix_NiPrunedFp32Fix/resolve/main/chilloutmix_NiPrunedFp32Fix.safetensors",
            "size": "4.27GB"
        }
    ]

    for model in model_urls:
        print(f"⬇️ {model['name']} ({model['size']}) をダウンロード中...")
        output_path = f"{model_dir}/{model['filename']}"

        # aria2cでダウンロード
        result = subprocess.run([
            'aria2c',
            '-c', '-x16', '-s16',
            '--check-certificate=false',
            '--summary-interval=10',
            '--max-tries=5',
            '--retry-wait=3',
            model['url'],
            '-d', model_dir,
            '-o', model['filename']
        ], capture_output=True, text=True)

        if result.returncode != 0:
            print(f"❌ {model['name']}のダウンロードに失敗")
            print("🔄 代替URLを試行中...")

            # 代替URL
            alt_urls = [
                f"https://huggingface.co/MarWinWinWinWin/ChillOutMix/resolve/main/{model['filename']}",
                f"https://huggingface.co/balapapapa/chilloutmix/resolve/main/{model['filename']}",
                f"https://huggingface.co/samle/sd-webui-models/resolve/main/{model['filename']}"
            ]

            success = False
            for alt_url in alt_urls:
                print(f"🔄 代替URL試行: {alt_url.split('/')[-3]}")
                result = subprocess.run([
                    'aria2c',
                    '-c', '-x8', '-s8',
                    '--check-certificate=false',
                    '--max-tries=3',
                    alt_url,
                    '-d', model_dir,
                    '-o', model['filename']
                ], capture_output=True, text=True)

                if result.returncode == 0 and os.path.exists(output_path):
                    success = True
                    break

            if not success:
                print("❌ すべての代替URLでダウンロードに失敗")
                print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
                print("🚨 手動ダウンロード手順:")
                print("1. https://huggingface.co/naonovn/chilloutmix_NiPrunedFp32Fix")
                print("2. chilloutmix_NiPrunedFp32Fix.safetensors をダウンロード")
                print("3. Google Colabにアップロードして以下に配置:")
                print(f"   {model_dir}/{model['filename']}")
                print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
                continue

        # ファイル検証
        if os.path.exists(output_path):
            file_size = os.path.getsize(output_path)
            if file_size > 1000000000:  # 1GB以上
                print(f"✅ {model['name']} ダウンロード完了 ({file_size / (1024*1024*1024):.2f} GB)")
            else:
                print(f"⚠️ {model['name']} ファイルサイズが小さすぎます ({file_size} bytes)")
        else:
            print(f"❌ {model['name']} ファイルが見つかりません")

# ChilloutMixモデルのダウンロード実行
download_chilloutmix_model()

#────────────────────────────────────────────
# 6) VAE (変分オートエンコーダー) をダウンロード
#────────────────────────────────────────────

print("\n🎨 VAEモデルをダウンロード中...")

vae_dir = '/content/stable-diffusion-webui/models/VAE'
os.makedirs(vae_dir, exist_ok=True)

# 高品質なVAEモデル
vae_models = [
    {
        "name": "vae-ft-mse-840000-ema-pruned.safetensors",
        "url": "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors"
    }
]

for vae in vae_models:
    print(f"⬇️ {vae['name']} をダウンロード中...")
    result = subprocess.run([
        'aria2c', '-c', '-x8', '-s8',
        vae['url'],
        '-d', vae_dir
    ], capture_output=True, text=True)

    if result.returncode == 0:
        print(f"✅ {vae['name']} ダウンロード完了")
    else:
        print(f"⚠️ {vae['name']} ダウンロード失敗")

#────────────────────────────────────────────
# 7) ControlNet モデルをダウンロード
#────────────────────────────────────────────

print("\n🎮 ControlNet モデルをダウンロード中...")

controlnet_dir = '/content/stable-diffusion-webui/extensions/sd-webui-controlnet/models'
os.makedirs(controlnet_dir, exist_ok=True)

# ControlNet モデル
controlnet_models = [
    {
        "name": "control_v11p_sd15_openpose.pth",
        "url": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth"
    },
    {
        "name": "control_v11f1p_sd15_depth.pth",
        "url": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth"
    },
    {
        "name": "control_v11p_sd15_canny.pth",
        "url": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth"
    }
]

for model in controlnet_models:
    print(f"⬇️ {model['name']} をダウンロード中...")
    result = subprocess.run([
        'aria2c', '-c', '-x4', '-s4',
        model['url'],
        '-d', controlnet_dir
    ], capture_output=True, text=True)

    if result.returncode == 0:
        print(f"✅ {model['name']} ダウンロード完了")
    else:
        print(f"⚠️ {model['name']} ダウンロード失敗（スキップ）")

#────────────────────────────────────────────
# 8) 必要ライブラリをインストール
#────────────────────────────────────────────

print("\n📚 追加ライブラリをインストール中...")

# xFormersの問題を回避するため、互換性のあるバージョンをインストール
print("🔧 xFormersの互換性問題を修正中...")
!pip uninstall -y xformers
!pip install -q xformers==0.0.20 --no-deps

# 必要なライブラリをインストール
!pip install -q open_clip_torch transformers timm
!pip install -q opencv-python
!pip install -q basicsr facexlib realesrgan gfpgan  # 顔修復用依存関係

print("✅ xFormers修正完了")

#────────────────────────────────────────────
# 9) 設定ファイルを作成
#────────────────────────────────────────────

print("⚙️ ChilloutMix用設定ファイルを作成中...")

# config.jsonを作成（ChilloutMix最適化設定）
config_content = """{
    "samples_save": true,
    "samples_format": "png",
    "samples_filename_pattern": "",
    "save_images_add_number": true,
    "grid_save": true,
    "grid_format": "png",
    "grid_extended_filename": false,
    "grid_only_if_multiple": true,
    "grid_prevent_empty_spots": false,
    "n_rows": -1,
    "enable_pnginfo": true,
    "save_txt": false,
    "save_images_before_face_restoration": true,
    "save_images_before_highres_fix": false,
    "save_images_before_color_correction": false,
    "jpeg_quality": 95,
    "webp_lossless": false,
    "export_for_4chan": true,
    "img_downscale_threshold": 4.0,
    "target_side_length": 4000,
    "img_max_size_mp": 200,
    "use_original_name_batch": true,
    "use_upscaler_name_as_suffix": false,
    "save_selected_only": true,
    "do_not_add_watermark": false,
    "temp_dir": "",
    "clean_temp_dir_at_start": false,
    "CLIP_stop_at_last_layers": 2,
    "upscaling_max_images_in_cache": 5,
    "face_restoration_model": "GFPGAN",
    "code_former_weight": 0.5,
    "face_restoration_unload": false
}"""

with open('/content/stable-diffusion-webui/config.json', 'w') as f:
    f.write(config_content)

# styles.csvを作成（ChilloutMix用プリセット）
styles_content = """name,prompt,negative_prompt
"ChilloutMix Portrait","(best quality, masterpiece:1.2), (realistic, photo-realistic:1.37), ultra detailed, 1 girl, cute, solo, beautiful detailed sky, detailed cafe, night, sitting, dating, (nose blush), (smile:1.15), (closed mouth) small breasts, beautiful detailed eyes, (collared shirt:1.1), bowtie, pleated skirt, (short hair:1.2), floating hair NovaFrogStyle","(worst quality, low quality:1.4), (realistic, lip, nose, tooth, rouge, lipstick, eyeshadow:1.0), (dusty sunbeams:1.0),, (abs, muscular, rib:1.0), (depth of field, bokeh, blurry, film grain, chromatic aberration, lens flare:1.0), (1boy, abs, muscular, rib:1.0), greyscale, monochrome, dusty sunbeams, trembling, motion lines, motion blur, emphasis lines, text, title, logo, signature"
"ChilloutMix Realistic","(8k, RAW photo, best quality, masterpiece:1.2), (realistic, photo-realistic:1.37), professional lighting, photon mapping, radiosity, physically-based rendering, cinematic lighting, intricate, High Detail, Sharp focus, dramatic, photorealistic painting","(worst quality, low quality:1.4), (realistic, lip, nose, tooth, rouge, lipstick, eyeshadow:1.0), (dusty sunbeams:1.0), (abs, muscular, rib:1.0), (depth of field, bokeh, blurry, film grain, chromatic aberration, lens flare:1.0), text, title, logo, signature"
"ChilloutMix Asian Beauty","(masterpiece, best quality), (realistic:1.3), finely detailed beautiful eyes and detailed face, masterpiece, best quality, (extremely detailed CG unity 8k wallpaper), most beautiful artwork in the world, professional majestic oil painting, intricate, High Detail, Sharp focus, dramatic, photorealistic, (asian girl), beautiful face, high detail eyes, elegant","(worst quality, low quality:1.4), multiple people, logo, text, badhandv4, (bad anatomy), (inaccurate limb), bad composition, inaccurate eyes, extra digit, fewer digits, (extra arms)"
"ChilloutMix Studio","studio lighting, professional photography, (masterpiece, best quality), (realistic:1.3), finely detailed beautiful eyes and detailed face, intricate High Detail, Sharp focus, dramatic, (upper body:1.2), looking at viewer","(worst quality, low quality:1.4), (depth of field, bokeh, blurry:1.4), (greyscale, monochrome:1.0), text, title, logo, signature"
"""

with open('/content/stable-diffusion-webui/styles.csv', 'w') as f:
    f.write(styles_content)

print("✅ 設定ファイル作成完了")

#────────────────────────────────────────────
# 10) 起動前の最終チェック
#────────────────────────────────────────────

print("\n" + "="*60)
print("🎭 ChilloutMix セットアップ完了サマリー:")
print("="*60)

# モデルファイルの確認
model_dir = "/content/stable-diffusion-webui/models/Stable-diffusion"
model_files = [f for f in os.listdir(model_dir) if f.endswith(('.safetensors', '.ckpt'))]

print(f"📁 インストールされたモデル:")
for model_file in model_files:
    file_path = os.path.join(model_dir, model_file)
    file_size = os.path.getsize(file_path) / (1024*1024*1024)
    print(f"   ✅ {model_file} ({file_size:.2f} GB)")

print(f"\n🔌 インストールされた拡張機能:")
ext_dir = "/content/stable-diffusion-webui/extensions"
extensions = [d for d in os.listdir(ext_dir) if os.path.isdir(os.path.join(ext_dir, d))]
for ext in extensions:
    print(f"   ✅ {ext}")

print(f"\n🎯 ChilloutMix 推奨設定:")
print("   📸 モデル: ChilloutMix-Ni")
print("   📐 解像度: 512x768 または 512x512")
print("   🎛️ CFG Scale: 7-12")
print("   🔄 Steps: 20-30")
print("   🎲 Sampler: DPM++ 2M Karras または Euler a")
print("   🎨 CLIP Skip: 2")

print(f"\n✨ ChilloutMix 特徴:")
print("   • リアルな人物画像生成に特化")
print("   • アジア系女性のポートレートで高品質")
print("   • 写実的な肌質・髪質表現")
print("   • スタジオ撮影風の画像生成")

print(f"\n💡 推奨プロンプト例:")
print('   • "1girl, portrait, realistic, beautiful detailed eyes"')
print('   • "professional photography, studio lighting, asian girl"')
print('   • "masterpiece, best quality, photorealistic, upper body"')

print(f"\n⚠️ Google Colab使用時の注意事項:")
print("   • メモリ不足エラーが出た場合は解像度を512x512に下げる")
print("   • 長時間使用すると接続が切れる場合がある")
print("   • 高品質画像は時間がかかる場合がある")

print(f"\n🔧 エラー対策済み:")
print("   • xFormersバージョン不整合を修正")
print("   • GFPGAN依存関係エラーを解決")
print("   • メモリ効率を改善（--lowvram使用）")
print("   • CUDA最適化設定を追加")

print("\n" + "="*60)

#────────────────────────────────────────────
# 11) Web UI を起動
#────────────────────────────────────────────

print("🚀 ChilloutMix Web UIを起動中...")
print("⏳ 初回起動には数分かかる場合があります...")
print("\n📱 起動完了後、共有URLが表示されます")

%cd /content/stable-diffusion-webui

# 環境変数を設定してxFormersの問題を完全に回避（ナレッジと同じ設定）
import os
os.environ['XFORMERS_FORCE_DISABLE_TRITON'] = '1'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Web UIを起動（WAI-NSFWと同じ設定、xFormers問題回避版）
!python launch.py \
    --share \
    --skip-torch-cuda-test \
    --disable-console-progressbars \
    --opt-sdp-attention \
    --enable-insecure-extension-access \
    --lora-dir "/content/stable-diffusion-webui/models/Lora" \
    --no-half \
    --precision full \
    --medvram \
    --no-half-vae \
    --api \
    --cors-allow-origins="*"

print("\n🎉 ChilloutMix Web UI起動完了!")
print("🎭 リアルな人物画像生成をお楽しみください！")

🎭 ChilloutMix Stable Diffusion WebUI セットアップ開始
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🔄 環境をクリーンアップ中...
✅ 既存のStable Diffusion WebUIディレクトリを削除
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Driveマウント完了

📝 ChilloutMixについて:
• リアルな人物画像生成に特化したモデル
• アジア系女性の画像生成で高い品質を発揮
• 写実的なポートレート生成が得意
• ファイルサイズ: 約4.27GB

📦 必要なパッケージをインストール中...
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
⬇️ Stable Diffusion Web UIをクローン中...
/content
🔥 PyTorch + CUDA 12.1をインストール中...
/content/stable-diffusion-webui
🔌 拡張機能をインストール中...
/content/stabl